<a href="https://colab.research.google.com/github/TheGreemDark/LR3_ML/blob/main/LR3_ML_Part3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Цель Блокнота

Решение задачи классификации в scikit-learn с помощью метода опорных векторов

* Применение `LabelEncoding` для изменения целевой переменной
* Обучение класса `SVC` и анализ атрибутов
* оптимизация гиперпараметров с использованием подхода случайного поиска `RandomizedSearchCV`
* Анализ модели для разных данных
* Сохранение модели

# Import библиотек

In [ ]:
# @title Гигантский блок импорта  { display-mode: "form" }
import pandas as pd # Библиотека Pandas для работы с табличными данными
import numpy as np # библиотека Numpy для операций линейной алгебры и прочего
import matplotlib.pyplot as plt # библиотека Matplotlib для визуализации
import seaborn as sns # библиотека seaborn для визуализации
import numpy as np # библиотека Numpy для операций линейной алгебры и прочего

import plotly.graph_objects as go # Библиотека Plotly. Модуль "Graph Objects"
import plotly.express as px # Библиотека Plotly. Модуль "Express"

# предварительная обработка числовых признаков
from sklearn.preprocessing import MinMaxScaler# Импортируем нормализацию от scikit-learn
from sklearn.preprocessing import StandardScaler # Импортируем стандартизацию от scikit-learn
from sklearn.preprocessing import PowerTransformer  # Степенное преобразование от scikit-learn
# предварительная обработка категориальных признаков
from sklearn.preprocessing import OneHotEncoder# Импортируем One-Hot Encoding от scikit-learn
from sklearn.preprocessing import OrdinalEncoder# Импортируем Порядковое кодирование от scikit-learn
from sklearn.preprocessing import LabelEncoder# Импортируем LabelEncoder от scikit-learn

from sklearn.pipeline import Pipeline # Pipeline.Не добавить, не убавить

from sklearn.compose import ColumnTransformer # т.н. преобразователь колонок

from sklearn.base import BaseEstimator, TransformerMixin # для создания собственных преобразователей / трансформеров данных


from sklearn.model_selection import train_test_split#  функция разбиения на тренировочную и тестовую выборку
# в исполнении scikit-learn

from sklearn.model_selection import StratifiedKFold # при кросс-валидации разбиваем данные в пропорции целевой метки
from sklearn.model_selection import cross_validate # функция кросс-валидации от Scikit-learn

from sklearn.metrics import f1_score # f1-мера от Scikit-learn
from sklearn.metrics import classification_report # функция scikit-learn которая считает много метрик классификации

from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats
#from sklearn.utils.fixes import loguniform #выводилась ошибка при импорте
from scipy.stats import loguniform #заменил на данный импорт


In [ ]:
from google.colab import files  # чтобы загружать файлы в облако через проводник

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#Набор данных
**Оценка возможности открытия депозитного счета в банке**

Набор данных, собранный в рамках маркетинговой кампании португальского банка, целью которой было телефонное продвижение банковского депозита (продукта term deposit), включает результаты прямого обзвона клиентов и информацию о них, чтобы предсказать, согласится ли клиент открыть депозит.

В датасете более 45 тысяч записей, каждая строка — один контакт с клиентом.

Всего около 17 атрибутов разных типов (категориальные и числовые).

Целевой параметр для задачи классификации — переменная y (yes/no), которая показывает, согласился ли клиент открыть срочный депозит после контакта.

#Загрузка данных на Google Drive

In [ ]:
uploaded = files.upload() #запуск и выбор файла в проводнике

Saving bank-full.csv to bank-full (2).csv


#Считывание файла в DataFrame
Используется метод .read_csv(path,delimiter)

In [ ]:
df = pd.read_csv('/content/bank-full.csv', delimiter = ';') # Открытие загруженного файла, через полный путь к файлу (с именем)
df # В этом блокноте мы работаем с данными без дубликатов

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


Список числовых и категориальных данных

In [ ]:
cat_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'y']
num_columns = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

Подробное описание каждого из столбцов.

age — возраст клиента (число)

job — профессия клиента (например, "admin", "technician" и т.д.)

marital — семейное положение ("married", "single", "divorced")

education — уровень образования ("primary", "secondary", "tertiary")

default — наличие невыплаченного кредита (yes/no)

balance — текущий баланс на счёте клиента (в евро)

housing — наличие ипотечного кредита (yes/no)

loan — наличие личного кредита (yes/no)

contact — способ связи с клиентом ("cellular", "telephone", "unknown")

day — день месяца последнего контакта (число)

month — месяц последнего контакта (например, "jan", "feb" и т.д.)

duration — длительность последнего звонка в секундах (число)

campaign — количество контактов с клиентом за текущую кампанию (число)

pdays — количество дней после последнего контакта в предыдущей кампании (число; 999 означает, что контакта не было)

previous — количество контактов с клиентом в предыдущих кампаниях (число)

poutcome — результат предыдущей маркетинговой кампании ("success", "failure", "other", "unknown")

y — целевая переменная: отклик клиента на маркетинговую кампанию (yes/no)

Предварительная обработка из [LR1_ML](https://elearn.urfu.ru/pluginfile.php/965749/assignsubmission_file/submission_files/2111145/LR1_ML.ipynb?forcedownload=1)

Из предварительной обработки исключена колонка `y`, поскольку она является целевой меткой и будет обработана отдельно


In [ ]:
class QuantileReplacer(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.05):
        self.threshold = threshold
        self.quantiles = {}

    def fit(self, X, y=None):
        for col in X.select_dtypes(include='number'):
            low_quantile = X[col].quantile(self.threshold)
            high_quantile = X[col].quantile(1 - self.threshold)
            self.quantiles[col] = (low_quantile, high_quantile)
        return self

    def transform(self, X):
        X_copy = X.copy()
        for col in X.select_dtypes(include='number'):
            low_quantile, high_quantile = self.quantiles[col]
            rare_mask = ((X[col] < low_quantile) | (X[col] > high_quantile))
            if rare_mask.any():
                rare_values = X_copy.loc[rare_mask, col]
                replace_value = np.mean([low_quantile, high_quantile])
                if rare_values.mean() > replace_value:
                    X_copy.loc[rare_mask, col] = high_quantile
                else:
                    X_copy.loc[rare_mask, col] = low_quantile
        return X_copy

In [ ]:
class RareGrouper(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.05, other_value='Other'):
        self.threshold = threshold
        self.other_value = other_value
        self.freq_dict = {}

    def fit(self, X, y=None):
        for col in X.select_dtypes(include=['object']):
            freq = X[col].value_counts(normalize=True)
            self.freq_dict[col] = freq[freq >= self.threshold].index.tolist()
        return self

    def transform(self, X, y=None):
        X_copy = X.copy()
        for col in X.select_dtypes(include=['object']):
            X_copy[col] = X_copy[col].apply(lambda x: x if x in self.freq_dict[col] else self.other_value)
        return X_copy

In [ ]:
#Степенное преобразование
num_pipe_age_balance_duration_day = Pipeline([
    ('power', PowerTransformer())
])

num_age_balance_duration_day = ['age', 'balance', 'day', 'duration']

#Стандартизация
num_pipe_campaign_pdays_previous = Pipeline([
    ('scaler', StandardScaler())
])

num_campaign_pdays_previous = ['campaign', 'pdays', 'previous']

#one-hot кодирование
cat_pipe_category_job = Pipeline([
    ('encoder', OneHotEncoder(drop='if_binary', handle_unknown='ignore', sparse_output=False))

])

cat_category_job = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']

# Сделаем отдельно Pipeline с числовыми признаками
preprocessors_num = ColumnTransformer(transformers=[
    ('num_age_balance_duration_day', num_pipe_age_balance_duration_day, num_age_balance_duration_day),
    ('num_campaign_pdays_previous', num_pipe_campaign_pdays_previous, num_campaign_pdays_previous),
])

# и Pipeline со всеми признаками
preprocessors_all = ColumnTransformer(transformers=[
    ('num_age_balance_duration_day', num_pipe_age_balance_duration_day, num_age_balance_duration_day),
    ('num_campaign_pdays_previous', num_pipe_campaign_pdays_previous, num_campaign_pdays_previous),
    ('cat_category_job', cat_pipe_category_job, cat_category_job),
])

In [ ]:
# объединяем названия колонок в один список (важен порядок как в ColumnTransformer)
columns_num = np.hstack([num_age_balance_duration_day,
                   num_campaign_pdays_previous,
                    cat_category_job,
                        ])

In [ ]:
# @title Вспомогательные функции { display-mode: "form" }
def cross_validation (X, y, model, scoring, cv_rule):
    """Расчет метрик на кросс-валидации.
    Параметры:
    ===========
    model: модель или pipeline
    X: признаки
    y: истинные значения
    scoring: словарь метрик
    cv_rule: правило кросс-валидации
    """
    scores = cross_validate(model,X, y,
                      scoring=scoring, cv=cv_rule )
    print('Ошибка на кросс-валидации')
    DF_score = pd.DataFrame(scores)
    display(DF_score)
    print('\n')
    print(DF_score.mean()[2:])


def calculate_metric(model_pipe, X, y, metric = f1_score):
    """Расчет метрики.
    Параметры:
    ===========
    model_pipe: модель или pipeline
    X: признаки
    y: истинные значения
    metric: метрика (f1 - по умолчанию)
    """
    y_model = model_pipe.predict(X)
    return metric(y, y_model)


def analyse_model(model, X_train, y_train, X_val, y_val, metrics, metric_names, scoring_reg, cv_rule):

    for name, metric,  in zip(metric_names, metrics):
        print(name+ f" на тренировочной выборке: {calculate_metric(model, X_train, y_train, metric):.4f}")
        print(name+ f" на валидационной выборке: {calculate_metric(model, X_val, y_val, metric):.4f}")
    print('--.--')
    print(classification_report(y_val, model.predict(X_val_prep), target_names=Label.classes_))
    print('--.--')
    cross_validation (X_train, y_train,
                    model,
                    scoring_reg,
                    cv_rule)

In [ ]:
scoring_clf = {'ACC': 'accuracy',
           'F1': 'f1',
           'Precision': 'precision',
           'Recall': 'recall'}



cv_rule = StratifiedKFold(n_splits=5, shuffle= True, random_state = 42)

metrics = [f1_score]

metric_names = ['f1_score']

# Метод опорных векторов для классификации

In [ ]:
from sklearn.svm import SVC # Метод опорных векторов для классификации scikit-learn

**Считываем данные, разбиваем на тестовую и тренировочную**

In [ ]:
# Удаление целевой переменной отклик клиента на маркетинговую кампанию из признаков
X,y = df.drop(columns = ['y']), df['y']

## Приводим целевые метки к 0 и 1

Воспользуемся объектом `LabelEncoder()` из модуля `preprocessing`


In [ ]:
Label = LabelEncoder()
Label.fit(y) # задаем столбец, который хотим преобразовать
Label.classes_ # в аттрибуте .classes_ хранится информация "какой класс как шифруется"

array(['no', 'yes'], dtype=object)

`0` это 'no', а `1` это 'yes'

In [ ]:
target = Label.transform(y) # преобразуем и сохраняем в новую переменную

In [ ]:
target # здесь уже только 0 и 1

array([0, 0, 0, ..., 1, 0, 0])

Так как метод опорных векторов работает довольно медленно, поэтому возьмем для демонстрации не все данные, а примерно 1/3

In [ ]:
# разбиваем на тренировочную и валидационную
X_train, X_val, y_train, y_val = train_test_split(X[::3], target[::3],
                                                    test_size=0.3,
                                                    random_state=42)

# Оценка модели с использованием только числовых данных

**Преобразуем данные**

In [ ]:
# Сначала обучаем на тренировочных данных
X_train_prep = preprocessors_num.fit_transform(X_train)
# потом на валидационной
X_val_prep = preprocessors_num.transform(X_val)

**Обучаем модель**

In [ ]:
model = SVC(random_state = 42)

model.fit(X_train_prep, y_train) ;

Найденные опорные вектора

In [ ]:
model.support_

array([   12,    21,    23, ..., 10541, 10542, 10548], dtype=int32)

In [ ]:
model.support_vectors_

array([[ 1.52586291,  0.4225388 ,  0.22999113, ..., -0.24385011,
        -0.41345599, -0.29916069],
       [-0.97202543,  7.78100369, -0.1211817 , ...,  1.102943  ,
        -0.41345599, -0.29916069],
       [-1.70594755, -0.3823122 ,  0.11510497, ..., -0.24385011,
        -0.41345599, -0.29916069],
       ...,
       [-0.47844523,  1.48331093,  1.38778794, ...,  0.09284816,
        -0.41345599, -0.29916069],
       [-0.47844523, -0.34172958,  1.48551845, ...,  0.09284816,
         3.09394812,  0.21323517],
       [ 0.14929135,  1.46841851,  0.56340747, ..., -0.24385011,
        -0.41345599, -0.29916069]])

Для этой конфигурации модели было найдено столько опорных векторов

In [ ]:
len(model.support_)

2692

Анализ модели

In [ ]:
analyse_model(model,
              X_train_prep, y_train,
              X_val_prep, y_val,
              metrics, metric_names,
              scoring_clf, cv_rule)

f1_score на тренировочной выборке: 0.1657
f1_score на валидационной выборке: 0.1675
--.--
              precision    recall  f1-score   support

          no       0.89      0.99      0.94      3988
         yes       0.68      0.10      0.17       534

    accuracy                           0.89      4522
   macro avg       0.79      0.54      0.55      4522
weighted avg       0.87      0.89      0.85      4522

--.--
Ошибка на кросс-валидации


,fit_time,score_time,test_ACC,test_F1,test_Precision,test_Recall
0,1.096788,0.236034,0.884834,0.089888,0.571429,0.048780
1,1.030358,0.248178,0.883412,0.108696,0.517241,0.060729
2,1.105086,0.433566,0.886256,0.142857,0.606061,0.080972
3,1.792315,0.335278,0.885782,0.117216,0.615385,0.064777
4,1.460562,0.240407,0.886676,0.124542,0.629630,0.069106




test_ACC          0.885392
test_F1           0.116640
test_Precision    0.587949
test_Recall       0.064873
dtype: float64


Выводы по модели:

* Если сравнивать с логистической регрессией, то получается, что точность снизилась на 10%
* На кросс-валидации также точность уменьшилась

* При этом данных меньше

## Покрутим ядро

In [ ]:
#@title **Параметры метода опорных векторов** { run: "auto" }
#@markdown ### Константа регуляризации
C=175 #@param {type:"slider", min:25, max:250, step:25}

#@markdown ### Параметры Ядер
kernel = 'rbf'  #@param [ 'rbf' , 'linear', 'poly']{type:"string"}
coef0=1 #@param {type:"slider", min:0, max:5, step:0.5}
degree=5 #@param {type:"slider", min:1, max:5, step:1}
gamma=0.95 #@param {type:"slider", min:0.00, max:1, step:0.05}

if gamma == 0:
  gamma='auto'

model_kernel = SVC(kernel=kernel, C=C, gamma=gamma,
            degree=degree, coef0 = coef0)

In [ ]:
model_kernel.fit(X_train_prep, y_train);

In [ ]:
analyse_model(model_kernel,
              X_train_prep, y_train,
              X_val_prep, y_val,
              metrics, metric_names,
              scoring_clf, cv_rule)

f1_score на тренировочной выборке: 0.8781
f1_score на валидационной выборке: 0.3093
--.--
              precision    recall  f1-score   support

          no       0.91      0.94      0.92      3988
         yes       0.36      0.27      0.31       534

    accuracy                           0.86      4522
   macro avg       0.63      0.60      0.62      4522
weighted avg       0.84      0.86      0.85      4522

--.--
Ошибка на кросс-валидации


,fit_time,score_time,test_ACC,test_F1,test_Precision,test_Recall
0,6.853390,1.020711,0.862085,0.340136,0.384615,0.304878
1,5.885879,0.285085,0.863507,0.327103,0.386740,0.283401
2,6.803717,0.528700,0.872038,0.389140,0.441026,0.348178
3,5.736393,0.289597,0.871090,0.384615,0.435897,0.344130
4,6.454401,0.421248,0.863917,0.331002,0.387978,0.288618




test_ACC          0.866527
test_F1           0.354399
test_Precision    0.407251
test_Recall       0.313841
dtype: float64


Выводы по модели:

* на тренировочных было получено почти `87%` точности для rbf
* получено это было ценой переобучения, так как разница между точностью на тренировочной и валидационной выборках слишком большая для rbf
* при этом точность на валидационной выборке улучшилась до 30% для rbf по сравнению с предыдущим вариантом (17%)

## Поиск лучших гиперпараметров с помощью рандомизированного поиска

In [ ]:
C_range = loguniform(1e-2, 1e2)
gamma_range = loguniform(1e-2, 1e0)
C_range_poly = loguniform(1e-1, 1e1)
tuned_parameters = [{'kernel': ['rbf'], 'gamma': gamma_range,
                     'C': C_range,},
                    {'kernel': ['poly'], 'degree': [2,3,4,], 'C': C_range_poly, }]


n_iter_search = 20
SVС_search = RandomizedSearchCV(estimator = SVC(coef0=0.5), verbose = 3,
                          param_distributions=tuned_parameters ,
                          cv=StratifiedKFold(n_splits=5, shuffle = True,random_state=42),n_iter = n_iter_search)

SVС_search.fit(X_train_prep, y_train)

SVCbest=SVС_search.best_estimator_

SVCbest.fit(X_train_prep, y_train);

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END C=0.032369776232792205, gamma=0.032158514131621074, kernel=rbf;, score=0.883 total time=   1.6s
[CV 2/5] END C=0.032369776232792205, gamma=0.032158514131621074, kernel=rbf;, score=0.883 total time=   0.9s
[CV 3/5] END C=0.032369776232792205, gamma=0.032158514131621074, kernel=rbf;, score=0.883 total time=   0.9s
[CV 4/5] END C=0.032369776232792205, gamma=0.032158514131621074, kernel=rbf;, score=0.883 total time=   1.0s
[CV 5/5] END C=0.032369776232792205, gamma=0.032158514131621074, kernel=rbf;, score=0.883 total time=   0.9s
[CV 1/5] END C=3.8252482159320285, gamma=0.048488526082546536, kernel=rbf;, score=0.884 total time=   1.5s
[CV 2/5] END C=3.8252482159320285, gamma=0.048488526082546536, kernel=rbf;, score=0.883 total time=   1.4s
[CV 3/5] END C=3.8252482159320285, gamma=0.048488526082546536, kernel=rbf;, score=0.887 total time=   1.5s
[CV 4/5] END C=3.8252482159320285, gamma=0.048488526082546536, kernel=rb

In [ ]:
SVCbest

SVC(C=np.float64(2.1849804967882323), coef0=0.5,
    gamma=np.float64(0.8643144315608566))

In [ ]:
analyse_model(SVCbest,
              X_train_prep, y_train,
              X_val_prep, y_val,
              metrics, metric_names,
              scoring_clf, cv_rule)

f1_score на тренировочной выборке: 0.6441
f1_score на валидационной выборке: 0.2853
--.--
              precision    recall  f1-score   support

          no       0.90      0.98      0.94      3988
         yes       0.52      0.20      0.29       534

    accuracy                           0.88      4522
   macro avg       0.71      0.59      0.61      4522
weighted avg       0.86      0.88      0.86      4522

--.--
Ошибка на кросс-валидации


,fit_time,score_time,test_ACC,test_F1,test_Precision,test_Recall
0,1.760919,0.338882,0.888152,0.293413,0.556818,0.199187
1,1.734341,0.333688,0.881043,0.296919,0.481818,0.214575
2,1.733039,0.329524,0.879621,0.265896,0.464646,0.186235
3,1.762390,0.333640,0.896682,0.351190,0.662921,0.238866
4,2.150395,0.583608,0.887150,0.316092,0.539216,0.223577




test_ACC          0.886530
test_F1           0.304702
test_Precision    0.541084
test_Recall       0.212488
dtype: float64


По сравнению с предыдущим шагом, точность снизилась до 29% на валидационной выборке
Настройка параметров coef0 (1), С (175) и gamma (0,55) дала результаты хуже, чем в предыдущих попытках (при С=125, gamma=0,95 и coef0=1 точность на валидационной выборке была 25%), поэтому лучше оставить gamma=0,95

In [ ]:
SVCbest.support_vectors_.shape

(3569, 7)

In [ ]:
DF_reg=pd.DataFrame(SVС_search.cv_results_)
DF = DF_reg[['param_C','param_kernel','param_degree','param_gamma',
             'mean_test_score', 'std_test_score', 'rank_test_score']]
cm = sns.light_palette("seagreen", as_cmap=True)
hl = DF.sort_values(by = 'rank_test_score').style.background_gradient(cmap=cm)
hl

,param_C,param_kernel,param_degree,param_gamma,mean_test_score,std_test_score,rank_test_score
17,1.642553,rbf,nan,0.542373,0.886909,0.004113,1
12,8.877042,poly,3.000000,nan,0.886245,0.001943,2
7,6.712041,poly,3.000000,nan,0.886150,0.002348,3
9,1.167253,poly,3.000000,nan,0.885582,0.001224,4
10,38.110893,rbf,nan,0.081262,0.885297,0.003175,5
5,0.126928,poly,4.000000,nan,0.883781,0.001977,6
8,4.354151,poly,4.000000,nan,0.883497,0.004298,7
1,1.169436,poly,4.000000,nan,0.883307,0.003299,8
3,1.147667,poly,4.000000,nan,0.883117,0.003220,9
11,0.031518,rbf,nan,0.071818,0.883117,0.000219,10


- Лучший результат достигается с rbf ядром при умеренном C (1.6) и достаточно высокой gamma (0.54).
- Poly ядро даёт близкие результаты на степени 3 и параметре C от 1 до 9, но чуть уступает лучшей rbf-модели.
- Высокие значения C не дают улучшения, скорее наоборот снижают производительность.
- Лучше всего сфокусироваться на оптимизации rbf ядра с гаммой в районе 0.5 и небольшого значения C для достижения максимально возможного качества качества.

# Выводы:

* Для задачи классификации добавление нелинейности не дало эффекта улучшения, лучшие показатели были достигнуты с помощью логистической регрессии
* Если сравнивать показатели точности, то при оценке влияния категориальных признаков логической регрессии было получено значение 47% на валидационной выборке, с помощью метода опорных векторов удалось достичь максимальной точности, равной 30% на валидационной выборке
* При этом метод опорных векторов чувствителен к гиперпараметрам
    * важно не только выбрать хорошее ядро, но и нужные цифры к нему
    * радиальное ядро ( `rbf` ) может выдавать результат хуже чем полиномиальное ядро
    * возможно причина в малых значениях `gamma`, но за счет уменьшения значения `gamma` и увеличения значения `С` не получается достичь большей точности.